In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd

df = pd.read_pickle('../data/topic.pkl')

In [2]:
tokenizer = AutoTokenizer.from_pretrained("mdraw/german-news-sentiment-bert")
model = AutoModelForSequenceClassification.from_pretrained("mdraw/german-news-sentiment-bert")
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier("Dies ist ein schlechter Test.")

[{'label': 'negative', 'score': 0.988132655620575}]

In [3]:
df['contains_cdu'] = df['preprocessed'].str.contains('cdu')
df['contains_csu'] = df['preprocessed'].str.contains('csu')
df['contains_fdp'] = df['preprocessed'].str.contains('fdp')
df['contains_gr'] = df['preprocessed'].str.contains('grüne') | df['preprocessed'].str.contains('grünen')
df['contains_li'] = df['preprocessed'].str.contains('linke') | df['preprocessed'].str.contains('linken')
df['contains_afd'] = df['preprocessed'].str.contains('afd') | df['preprocessed'].str.contains('afg')
df['contains_spd'] = df['preprocessed'].str.contains('spd')

In [4]:
ind = 4
subset = 'contains_gr'

teststring = df.loc[df[subset]].iloc[ind]['transcript']
print(f'String: {teststring}')
print(f'Classification: {classifier(teststring)}')

String: bewegungen durch kämpfe auf den straßen im rahmen der proteste dass wir es sind die die forderung stellen und auch fordern müssen wir müssen die fragen richtig stellen und wir müssen auch die richtigen antworten geben darauf danket über meine nächste frage geht ein juhu kurz vor der bundestagswahl gab es ja noch mal einen großen streit tag bei future nun sind die grünen in der bundesregierung wenn es nach der ampel geht soll die klimakatastrophe gelöst werden durch innovationen in industrie technologie und wirtschaft unterstützt durch staatliche unterstützung unterstützungsprogramme es wird immer mehr produziert ohne dabei auf die bedürfnisse der bevölkerung einzugehen so sollen zb bis 20.30 mindestens 15 millionen e-autos auf deutschen straßen rollen ein radikales umdenken in sachen klimaschutz blieb wie zu erwarten war aus viele jugendliche sehen in den grünen jedoch ein fortschritt wie geht 
Classification: [{'label': 'neutral', 'score': 0.5348684191703796}]
